# 청구입금정보 EDA

In [1]:
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# 설치된 폰트 목록을 갱신하고, 'NanumGothic' 폰트를 기본 폰트로 설정합니다.
font_path = 'C:/Windows/Fonts/NanumGothic.ttf' 
font_name = fm.FontProperties(fname=font_path).get_name()
plt.rc('font', family=font_name)
plt.rcParams['axes.unicode_minus'] = False

import sys
sys.path.append('../utils')

In [2]:
from sklearn.pipeline import Pipeline
from sklearn import set_config
from preprocessing import (
    DateElapsedTransformer, NumericFeaturePreprocessor, ObjectFeaturePreprocessor, NumericTypeOptimizer,
)

pre_optimizer = NumericTypeOptimizer(mode='pre')
date_transformer = DateElapsedTransformer()
numeric_transformer = NumericFeaturePreprocessor()
object_transformer = ObjectFeaturePreprocessor()
post_optimizer = NumericTypeOptimizer(mode='post')

preprocessing_pipeline = Pipeline([
    ('pre_optimizer', pre_optimizer),
    ('object', object_transformer),
    ('date', date_transformer),
    ('numeric', numeric_transformer),
    ('post_optimizer', post_optimizer),
])

# Set display configuration to visualize the pipeline
set_config(display='diagram')

# Display the pipeline
preprocessing_pipeline

Pipeline(steps=[('pre_optimizer', NumericTypeOptimizer(mode='pre')),
                ('object',
                 ObjectFeaturePreprocessor(exclude_columns=['ID', 'Segment'])),
                ('date',
                 DateElapsedTransformer(base_date=Timestamp('2019-01-01 00:00:00'),
                                        exclude_columns=['ID', 'Segment',
                                                         '기준년월'])),
                ('numeric',
                 NumericFeaturePreprocessor(exclude_columns=['기준년월'])),
                ('post_optimizer', NumericTypeOptimizer())])

In [3]:
from sklearn.preprocessing import LabelEncoder
from data_loader import load_data

bill_df, bill_test_df = load_data("4.청구입금정보")

X = bill_df.drop(columns=['ID', '기준년월'])
X = preprocessing_pipeline.fit_transform(X)
X.info()

✅ File: ../../dataset/train/4.청구입금정보\201807_train_청구정보.parquet Completed!
✅ File: ../../dataset/train/4.청구입금정보\201808_train_청구정보.parquet Completed!
✅ File: ../../dataset/train/4.청구입금정보\201809_train_청구정보.parquet Completed!
✅ File: ../../dataset/train/4.청구입금정보\201810_train_청구정보.parquet Completed!
✅ File: ../../dataset/train/4.청구입금정보\201811_train_청구정보.parquet Completed!
✅ File: ../../dataset/train/4.청구입금정보\201812_train_청구정보.parquet Completed!
🔹 Shape : (2400000, 46)

✅ File: ../../dataset/test/4.청구입금정보\201807_test_청구정보.parquet Completed!
✅ File: ../../dataset/test/4.청구입금정보\201808_test_청구정보.parquet Completed!
✅ File: ../../dataset/test/4.청구입금정보\201809_test_청구정보.parquet Completed!
✅ File: ../../dataset/test/4.청구입금정보\201810_test_청구정보.parquet Completed!
✅ File: ../../dataset/test/4.청구입금정보\201811_test_청구정보.parquet Completed!
✅ File: ../../dataset/test/4.청구입금정보\201812_test_청구정보.parquet Completed!
🔹 Shape : (600000, 46)
Numeric Type Optimizer Transforming...
🧠 [mode=pre] 메모리 최적화: 1926.41 MB → 14

## Feature Selection

In [19]:
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns

def get_feature_importance(X, y, importance_type='gain', plot_top_n=30):
    """
    LightGBM을 이용해 Feature Importance를 계산하고, 상위 feature를 시각화합니다.
    
    Parameters:
    - X (pd.DataFrame): feature 데이터
    - y (pd.Series or np.array): target 데이터
    - importance_type (str): 'gain' 또는 'split' (중요도 기준)
    - plot_top_n (int): 중요도가 높은 feature 몇 개를 plot할지
    - random_state (int): seed 고정

    Returns:
    - feature_importance_df (pd.DataFrame): feature importance 데이터프레임
    - model (lgb.Booster): 학습된 LightGBM 모델
    """

    # LightGBM 데이터셋 구성
    lgb_train = lgb.Dataset(X, label=y)

    # LightGBM 기본 파라미터 (가볍게 학습)
    params = {
        'objective': 'multiclass',   # 다중 클래스 분류
        'num_class': 5,  # 클래스 수 자동 지정
        'boosting_type': 'gbdt',
        'metric': 'multi_logloss',   # 다중 클래스 로지 손실
        'verbosity': -1,
        'seed': 42,
        'learning_rate': 0.05,
        'num_leaves': 31,
        'min_data_in_leaf': 20
    }

    # 빠르게 학습 (에포크 100회로 제한)
    model = lgb.train(params, lgb_train, num_boost_round=100)

    # Feature Importance 가져오기
    importance = model.feature_importance(importance_type=importance_type)
    feature_names = X.columns

    feature_importance_df = pd.DataFrame({
        'feature': feature_names,
        'importance': importance
    }).sort_values('importance', ascending=False)

    # 시각화
    top_features = feature_importance_df.head(plot_top_n)
    
    plt.figure(figsize=(10, max(6, plot_top_n // 2)))
    plt.barh(top_features['feature'][::-1], top_features['importance'][::-1])
    plt.xlabel('Importance')
    plt.title(f'Top {plot_top_n} Feature Importance ({importance_type})')
    plt.tight_layout()
    plt.show()

    return feature_importance_df, model

In [20]:
X = bill_df.drop(columns=['ID', '기준년월', 'Segment'])

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y = le.fit_transform(bill_df['Segment'])

importance_df, model = get_feature_importance(X, y, importance_type='gain', plot_top_n=30)

- 포인트 적립/사용과 청구금액이 고객의 활동성과 소비 규모를 가장 잘 반영한다고 모델이 판단

In [21]:
def select_top_features(importance_df, X, top_percent=0.3):
    """
    중요도 데이터프레임에서 상위 top_percent 비율만큼 feature를 선택하여 데이터셋 추출

    Parameters:
    - importance_df (pd.DataFrame): feature와 importance를 가진 데이터프레임
    - X (pd.DataFrame): 원본 feature 데이터셋
    - top_percent (float): 상위 몇 %를 선택할지 (기본 30%)

    Returns:
    - X_top (pd.DataFrame): 상위 feature만 선택된 데이터프레임
    - selected_features (list): 선택된 feature 리스트
    """
    n_features = int(len(importance_df) * top_percent)
    selected_features = importance_df['feature'].iloc[:n_features].tolist()
    X_top = X[selected_features]
    return X_top, selected_features

In [22]:
X_top, selected_features = select_top_features(importance_df, X, top_percent=0.3)
print(f"🔹 선택된 feature 수: {len(selected_features)}")
print(f"🔹 선택된 feature: {selected_features}")

🔹 선택된 feature 수: 12
🔹 선택된 feature: ['포인트_포인트_건별_R3M', '청구금액_R3M', '포인트_이용포인트_R12M', '포인트_이용포인트_R3M', '청구금액_R6M', '연체건수_R6M', '대표결제일', '혜택수혜금액', '선결제건수_R6M', '혜택수혜금액_R3M', '할인금액_R3M', '포인트_적립포인트_R12M']


In [23]:
def plot_feature_correlation(X_top, threshold=0.8):
    """
    상위 feature들 간의 상관계수 행렬을 시각화하고, 
    threshold 이상인 feature쌍을 출력

    Parameters:
    - X_top (pd.DataFrame): 상위 feature 데이터프레임
    - threshold (float): 높은 상관관계로 간주할 기준
    """
    # 수치치형 feature만 선택
    numeric_features = X_top.select_dtypes(include=['number'])
    corr_matrix = numeric_features.corr()
    print(" 상관계수 행렬 생성 완료..!")

    # 상관관계 히트맵 그리기
    plt.figure(figsize=(12, 10))
    sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt='.2f')
    plt.title('Correlation Matrix of Top Features')
    plt.show()

    # threshold 이상인 feature 쌍 출력
    high_corr_pairs = []
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > threshold:
                high_corr_pairs.append((corr_matrix.columns[i], corr_matrix.columns[j], corr_matrix.iloc[i, j]))

    if high_corr_pairs:
        print(f"\n상관계수 {threshold} 이상인 feature 쌍:")
        for f1, f2, corr in sorted(high_corr_pairs, key=lambda x: -abs(x[2])):
            print(f"🔹{f1} vs {f2}: 상관계수 = {corr:.3f}")
    else:
        print(f"\n상관계수 {threshold} 이상인 feature 쌍이 없습니다.")
    
    return high_corr_pairs

In [24]:
# 상관계수 0.8 이상인 feature 쌍을 시각화
high_corr_pairs = plot_feature_correlation(X_top, threshold=0.8)

 상관계수 행렬 생성 완료..!



상관계수 0.8 이상인 feature 쌍:
🔹혜택수혜금액_R3M vs 혜택수혜금액: 상관계수 = 0.957
🔹청구금액_R6M vs 청구금액_R3M: 상관계수 = 0.943
🔹포인트_이용포인트_R3M vs 포인트_이용포인트_R12M: 상관계수 = 0.920
🔹포인트_적립포인트_R12M vs 포인트_포인트_건별_R3M: 상관계수 = 0.889


In [26]:
high_corr_pairs = sorted(high_corr_pairs, key=lambda x: x[2], reverse=True)
high_corr_pairs

[('혜택수혜금액_R3M', '혜택수혜금액', np.float64(0.9565425347149744)),
 ('청구금액_R6M', '청구금액_R3M', np.float64(0.9425180702245619)),
 ('포인트_이용포인트_R3M', '포인트_이용포인트_R12M', np.float64(0.9195936483979517)),
 ('포인트_적립포인트_R12M', '포인트_포인트_건별_R3M', np.float64(0.8893924567172236))]

In [27]:
features_to_drop = set()
for f1, f2, corr in high_corr_pairs:
    if corr > 0.9:
        if f1 not in features_to_drop and f2 not in features_to_drop:
            features_to_drop.add(f1)

features_to_drop

{'청구금액_R6M', '포인트_이용포인트_R3M', '혜택수혜금액_R3M'}

In [28]:
for col in features_to_drop:
    X_top = X_top.drop(columns=[col])

X_top.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2400000 entries, 0 to 2399999
Data columns (total 9 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   포인트_포인트_건별_R3M  int16 
 1   청구금액_R3M        uint32
 2   포인트_이용포인트_R12M  uint32
 3   연체건수_R6M        uint8 
 4   대표결제일           uint8 
 5   혜택수혜금액          uint16
 6   선결제건수_R6M       uint8 
 7   할인금액_R3M        uint16
 8   포인트_적립포인트_R12M  uint32
dtypes: int16(1), uint16(2), uint32(3), uint8(3)
memory usage: 48.1 MB


In [29]:
filtered_bill_df = bill_df[['ID', '기준년월'] + X_top.columns.tolist()]
filtered_bill_df.to_parquet('../dataset/train/bill.parquet', index=False)

filtered_bill_test_df = bill_test_df[['ID', '기준년월'] + X_top.columns.tolist()]
filtered_bill_test_df.to_parquet('../dataset/test/bill.parquet', index=False)